# User Satisfaction Analysis

## Task 1 - User Overview Analysis

Understand the dataset and identify missing values & outliers using visual and quantitative methods.

### tasks:
- Identify the top 10 handsets used by customers.
- Identify the top 3 handset manufacturers.
- Identify the top 5 handsets per top 3 handset manufacturer.
- Provide interpretation and recommendations to marketing teams.


In [1]:

# Import necessary libraries and modules
import os
import sys
import warnings
import pandas as pd

# Add parent directory to path to import local modules
sys.path.insert(0, os.path.dirname(os.getcwd()))
from scripts.connect_db import conn_db 
from scripts.user_overview import UserOverview

# Suppress FutureWarnings
warnings.simplefilter(action='ignore', category=FutureWarning)

## Connecting to a PostgreSQL Database

In [2]:
db = conn_db(
    database='TellCo_db',
    user='postgres',
    password='SH36@jit',
    host='localhost',
    port='5432'
)
db.connect()

Connected to TellCo_db database successfully.


### Fetching Data and Initializing Data Analysis

In [3]:
# Fetch data as a DataFrame
df = db.fetch_data("SELECT * FROM xdr_data;")
db.disconnect()
df.head()

TellCo_db connection closed.


,Bearer Id,Start,Start ms,End,End ms,Dur. (s),IMSI,MSISDN/Number,IMEI,Last Location Name,...,Youtube DL (Bytes),Youtube UL (Bytes),Netflix DL (Bytes),Netflix UL (Bytes),Gaming DL (Bytes),Gaming UL (Bytes),Other DL (Bytes),Other UL (Bytes),Total UL (Bytes),Total DL (Bytes)
0,1.311448e+19,4/4/2019 12:01,770.0,4/25/2019 14:35,662.0,1823652.0,2.082014e+14,3.366496e+10,3.552121e+13,9.16456699548519E+015,...,15854611.0,2501332.0,8198936.0,9656251.0,278082303.0,14344150.0,171744450.0,8814393.0,36749741.0,308879636.0
1,1.311448e+19,4/9/2019 13:04,235.0,4/25/2019 8:15,606.0,1365104.0,2.082019e+14,3.368185e+10,3.579401e+13,L77566A,...,20247395.0,19111729.0,18338413.0,17227132.0,608750074.0,1170709.0,526904238.0,15055145.0,53800391.0,653384965.0
2,1.311448e+19,4/9/2019 17:42,1.0,4/25/2019 11:58,652.0,1361762.0,2.082003e+14,3.376063e+10,3.528151e+13,D42335A,...,19725661.0,14699576.0,17587794.0,6163408.0,229584621.0,395630.0,410692588.0,4215763.0,27883638.0,279807335.0
3,1.311448e+19,4/10/2019 0:31,486.0,4/25/2019 7:36,171.0,1321509.0,2.082014e+14,3.375034e+10,3.535661e+13,T21824A,...,21388122.0,15146643.0,13994646.0,1097942.0,799538153.0,10849722.0,749039933.0,12797283.0,43324218.0,846028530.0
4,1.311448e+19,4/12/2019 20:10,565.0,4/25/2019 10:40,954.0,1089009.0,2.082014e+14,3.369980e+10,3.540701e+13,D88865A,...,15259380.0,18962873.0,17124581.0,415218.0,527707248.0,3529801.0,550709500.0,13910322.0,38542814.0,569138589.0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150001 entries, 0 to 150000
Data columns (total 55 columns):
 #   Column                                    Non-Null Count   Dtype  
---  ------                                    --------------   -----  
 0   Bearer Id                                 149010 non-null  float64
 1   Start                                     150000 non-null  object 
 2   Start ms                                  150000 non-null  float64
 3   End                                       150000 non-null  object 
 4   End ms                                    150000 non-null  float64
 5   Dur. (s)                                  150000 non-null  float64
 6   IMSI                                      149431 non-null  float64
 7   MSISDN/Number                             148935 non-null  float64
 8   IMEI                                      149429 non-null  float64
 9   Last Location Name                        148848 non-null  object 
 10  Avg RTT DL (ms)     

 ### Initial Data Analysis Insights: 📊
  - Dataset contains 150,001 records with 55 columns 📝
  - Most columns are numeric (50 float64) with only 5 object (string) columns 🔢
  - Several columns have missing values, particularly: ⚠️
    - TCP DL/UL Retrans. Vol (~60% missing) 📉
    - HTTP DL/UL Bytes (~55% missing) 📊
    - Volume-based time metrics (60-87% missing) ⏱️
  - All application traffic columns (Social Media, Google, Email etc.) have complete data ✅
  - Core user identifiers (MSISDN, IMSI, IMEI) have ~1% missing values 🆔
  - Key metrics like duration, throughput and total bytes are mostly complete 📈


## Identifying Columns with High Percentage of Missing Values

In [5]:

# Calculate the percentage of missing values in each column
missing_values = df.isnull().sum() / len(df)

# Filter the columns where more than 10% of the values are missing
high_missing_values = missing_values[missing_values > 0.1]

print(high_missing_values)

Avg RTT DL (ms)                             0.185525
Avg RTT UL (ms)                             0.185412
TCP DL Retrans. Vol (Bytes)                 0.587636
TCP UL Retrans. Vol (Bytes)                 0.644322
HTTP DL (Bytes)                             0.543156
HTTP UL (Bytes)                             0.545396
Nb of sec with 125000B < Vol DL             0.650249
Nb of sec with 1250B < Vol UL < 6250B       0.619289
Nb of sec with 31250B < Vol DL < 125000B    0.623903
Nb of sec with 37500B < Vol UL              0.868354
Nb of sec with 6250B < Vol DL < 31250B      0.588776
Nb of sec with 6250B < Vol UL < 37500B      0.745615
dtype: float64


# Data Variables Analysis and Column Selection Strategy 📊

### Overview of Dataset Structure and Completeness

Our dataset contains 150,001 entries with 55 columns, consisting of:
- 50 numeric columns (float64)
- 5 categorical columns (object)

### Column Selection Strategy Based on Data Completeness

#### High Completeness (>99% non-null)
- Core identifiers (Bearer Id, IMSI, MSISDN/Number, IMEI)
- Timing data (Start, End, Duration)
- All application traffic metrics (Social Media, Google, Email, etc.)
- Total traffic metrics (Total UL/DL Bytes)
- Throughput percentages (DL/UL TP distributions)
- Device information (Handset Manufacturer, Type)

#### Moderate Completeness (80-99% non-null)
- Network performance metrics (Avg RTT DL/UL)

#### Low Completeness (<80% non-null)
- TCP retransmission volumes (~40% non-null)
- HTTP traffic metrics (~45% non-null)
- Detailed volume-based time metrics (~25-40% non-null)

### Selected Essential Columns

#### Core User/Session Identification (>99% complete)
- `Bearer Id`: Session tracking (99.3% complete)
- `Start`, `End`, `Dur. (s)`: Session timing (100% complete)
- `MSISDN/Number`: User ID (99.3% complete)
- `IMSI`, `IMEI`: Device ID (99.6% complete)

#### Network Performance Metrics
- `Avg Bearer TP DL/UL (kbps)`: Throughput (100% complete)
- `Avg RTT DL/UL (ms)`: Latency (~81% complete)
- `TCP Retrans. Vol`: Retained despite low completeness for QoS analysis

#### Application Usage (100% complete)
- Social Media, Google, Email traffic
- Streaming (YouTube, Netflix)
- Gaming traffic
- Other and Total traffic

#### Device Information (99.6% complete)
- `Handset Manufacturer`
- `Handset Type`

#### Network Quality Indicators (~99.5% complete)
- DL/UL throughput distribution percentages

### Columns Excluded Due to:

1. Low Completeness (<50%):
   - HTTP metrics
   - Volume-based time metrics

2. Redundancy:
   - Millisecond precision timestamps
   - Duplicate duration metrics

### Data Quality Summary
- Core metrics maintain >99% completeness
- Network performance metrics show varying completeness
- Application usage data is complete
- Device information highly reliable

This selection strategy prioritizes data completeness while maintaining comprehensive coverage of key performance and usage metrics.

In [6]:
# Initialize UserOverview class with our DataFrame
user_overv = UserOverview(df)

## Select Essential Columns for Analysis


In [7]:
# select relevent columns
select_data = user_overv.select_essential_columns()
select_data.head()

,Bearer Id,Start,End,Dur. (s),MSISDN/Number,IMSI,IMEI,Avg Bearer TP DL (kbps),Avg Bearer TP UL (kbps),Avg RTT DL (ms),...,Other DL (Bytes),Other UL (Bytes),Total DL (Bytes),Total UL (Bytes),Handset Manufacturer,Handset Type,DL TP < 50 Kbps (%),50 Kbps < DL TP < 250 Kbps (%),UL TP < 10 Kbps (%),10 Kbps < UL TP < 50 Kbps (%)
0,1.311448e+19,4/4/2019 12:01,4/25/2019 14:35,1823652.0,3.366496e+10,2.082014e+14,3.552121e+13,23.0,44.0,42.0,...,171744450.0,8814393.0,308879636.0,36749741.0,Samsung,Samsung Galaxy A5 Sm-A520F,100.0,0.0,100.0,0.0
1,1.311448e+19,4/9/2019 13:04,4/25/2019 8:15,1365104.0,3.368185e+10,2.082019e+14,3.579401e+13,16.0,26.0,65.0,...,526904238.0,15055145.0,653384965.0,53800391.0,Samsung,Samsung Galaxy J5 (Sm-J530),100.0,0.0,100.0,0.0
2,1.311448e+19,4/9/2019 17:42,4/25/2019 11:58,1361762.0,3.376063e+10,2.082003e+14,3.528151e+13,6.0,9.0,NaN,...,410692588.0,4215763.0,279807335.0,27883638.0,Samsung,Samsung Galaxy A8 (2018),100.0,0.0,100.0,0.0
3,1.311448e+19,4/10/2019 0:31,4/25/2019 7:36,1321509.0,3.375034e+10,2.082014e+14,3.535661e+13,44.0,44.0,NaN,...,749039933.0,12797283.0,846028530.0,43324218.0,undefined,undefined,100.0,0.0,100.0,0.0
4,1.311448e+19,4/12/2019 20:10,4/25/2019 10:40,1089009.0,3.369980e+10,2.082014e+14,3.540701e+13,6.0,9.0,NaN,...,550709500.0,13910322.0,569138589.0,38542814.0,Samsung,Samsung Sm-G390F,100.0,0.0,100.0,0.0


## Clean and Handle Missing Values


In [10]:
clean_data = user_overv.wrangle()
clean_data.head()

,Bearer Id,Start,End,Dur. (s),MSISDN/Number,IMSI,IMEI,Avg Bearer TP DL (kbps),Avg Bearer TP UL (kbps),Avg RTT DL (ms),...,Other DL (Bytes),Other UL (Bytes),Total DL (Bytes),Total UL (Bytes),Handset Manufacturer,Handset Type,DL TP < 50 Kbps (%),50 Kbps < DL TP < 250 Kbps (%),UL TP < 10 Kbps (%),10 Kbps < UL TP < 50 Kbps (%)
0,1.311448e+19,4/4/2019 12:01,4/25/2019 14:35,1823652.0,3.366496e+10,2.082014e+14,3.552121e+13,23.0,44.0,42.000000,...,171744450.0,8814393.0,308879636.0,36749741.0,Samsung,Samsung Galaxy A5 Sm-A520F,100.0,0.0,100.0,0.0
1,1.311448e+19,4/9/2019 13:04,4/25/2019 8:15,1365104.0,3.368185e+10,2.082019e+14,3.579401e+13,16.0,26.0,65.000000,...,526904238.0,15055145.0,653384965.0,53800391.0,Samsung,Samsung Galaxy J5 (Sm-J530),100.0,0.0,100.0,0.0
2,1.311448e+19,4/9/2019 17:42,4/25/2019 11:58,1361762.0,3.376063e+10,2.082003e+14,3.528151e+13,6.0,9.0,109.795706,...,410692588.0,4215763.0,279807335.0,27883638.0,Samsung,Samsung Galaxy A8 (2018),100.0,0.0,100.0,0.0
3,1.311448e+19,4/10/2019 0:31,4/25/2019 7:36,1321509.0,3.375034e+10,2.082014e+14,3.535661e+13,44.0,44.0,109.795706,...,749039933.0,12797283.0,846028530.0,43324218.0,undefined,undefined,100.0,0.0,100.0,0.0
4,1.311448e+19,4/12/2019 20:10,4/25/2019 10:40,1089009.0,3.369980e+10,2.082014e+14,3.540701e+13,6.0,9.0,109.795706,...,550709500.0,13910322.0,569138589.0,38542814.0,Samsung,Samsung Sm-G390F,100.0,0.0,100.0,0.0
